# Predicting SAP stock prices
## Using PyTorch and SageMaker

---


## Instructions

> **Note**: Code and Markdown cells can be executed using the **Shift+Enter** keyboard shortcut. In addition, a cell can be edited by typically clicking it (double-click for Markdown cells) or by pressing **Enter** while it is highlighted.

## General Outline

Recall the general outline for SageMaker projects using a notebook instance.

1. Part 1: Data preparation
  1. Downloading data
  2. Prepare the data
  3. Divide into training and testing set
  4. Scale/Normalize the data
  5. Save the data
2. Part 2: Local training and testing
  1. Define a network model
  1. Run training inside the notebook
  2. Get insights regarding training quality
3. Part 3: Use AWS for training and prediction
  1. Prepare custom code for 
    1. Neural network model
    2. Training
  2. Upload the processed data to S3
  3. Train the model on a separate GPU-optimized instance
  4. Deploy the trained model on a separate instance
  5. Use the model for predictions
  6. Cleanup


# Disclaimer

This is notebook is meant to demonstrate selected abilities of AWS in field of Machine Learning. 

Neural network used in this notebook is not optimized for prediction of stock prices. 
In fact, its predictions are very low quality. Predicting stock prices is a very difficult task! 
**Do not invest any money into stock based on these predictions**


# Part 1: Data Preparation

## Section 1.1: Downloading data

We use the historical data from a polish provider Stooq.


Visit https://stooq.com/q/d/?s=sap.de to fine-tune the data.

In [ ]:
import os
import glob
import math
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt

import torch
import torch.utils.data
import torch.optim as optim
from torch import nn

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import sagemaker
from sagemaker.pytorch import PyTorch


# Global settings
forecast_days = 1       # How many days to the future shall the stock price be predicted
                        # For example: forecasting 2 days to the future will use the data from last days
                        # to predict the price on a day 2 days ahead, but will not predict the price 1 day ahead

batch_size = 10
gradient_clip_maximum = 5


Selected commands similar to regular Linux commands can be used in the notebook.
These commands are called *line magics*. List of available magics looks like this:

In [ ]:
% lsmagic

Regular Linux shell command can be run as well by preceding them with exclamation mark: `!`

We use these tools to download the data from the source and store it in csv file

In [ ]:
%mkdir -p "data"
# Clean existing data if necessary
!rm -rf "data/*"

!curl -k -sS "https://stooq.com/q/d/l/?s=sap.de&i=d" > "data/data.raw.csv"
# i=d - interval daily (other: w,m,q,y - week, month, quarter, year)
# o=1100000 - options - binary mask 
# other...

We may want to start learning with simpler data, so here is the option to replace stock prices with `sin` signal

In [ ]:
def create_target_for_supervised_learning(src, target, shift=1):
    """
    This function shifts target forward by `shift` positions with respect to the src
    Example for shift=1:
    [a,b,c] [a,b,c] -> [a,b,c] [x,a,b]
    """
    src = pd.DataFrame(src)
    target = pd.DataFrame(target)
    
    src = src.iloc[:(shift * -1)]
    target = target.iloc[shift:]
    
    # Data was shifted and 'empty spaces' in form of NaNs have emerged
    # We may fill it with zeros:
    # src.fillna(0, inplace=True)
    # or interpolate
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
    src.interpolate(method='nearest', inplace=True)
    target.interpolate(method='nearest', inplace=True)
    
    src.reset_index(inplace=True, drop=True)
    target.reset_index(inplace=True, drop=True)

    return src, target

## Section 1.2: Preparing and Processing the data

Default data range used at stooq is: September 1994 until today.

Let's look at the data and discard any pieces of it that are not interesting for us.

In [ ]:
def read_stock_data(data_dir='data/'):  
    ymd_date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
    return pd.read_csv(data_dir+"data.raw.csv", parse_dates=['Date'], date_parser=ymd_date_parser)
   
    
def select_columns(raw):
    input_data = pd.DataFrame({
        "AvgPrice": raw['Low'] + (raw['High'] - raw['Low'])/2
    })
    target_data = input_data.copy()
    return input_data, target_data

# Utility function to plot data quickly
def plot_data(input_data, target_data, legend_1="left", legend_2="right", title="no title", first_days=None):
    plt.figure(figsize=(16,5))
    x_axis = range(0,len(input_data))
    plt.title(title)
    plt.plot(x_axis[:first_days], input_data[:first_days], 'b.-', label=legend_1)
    
    if target_data is not None:
        plt.plot(x_axis[:first_days], target_data[:first_days], 'r.-', label=legend_2)
    plt.legend(loc='best')
    plt.show()

In [ ]:
raw_data = read_stock_data()
print("Raw data imported from the source:\n ", raw_data.head(5))


We require columns of data containing the price of the stock. 
We calculate an average stock price based on the maximum and minimum from a given day.
Rest of the data is discarded. 

First column should contain input, whereas the second column contains the target.
The target column should be shifted forward by `forecast_days` in respect to the input column.

In [ ]:
input_data, target_data = select_columns(raw_data)
input_data, target_data = create_target_for_supervised_learning(input_data, target_data, forecast_days)

print("Shape of input:\n", input_data.shape)
print("Shape of target:\n", target_data.shape)

print("Head of input:\n", input_data.head(5))
print("Head of target:\n", target_data.head(5))

Plotting the data provides a chart depicting that the shift has succeeded. 
For any given day, the red curve represents the price of the stock from the next day.

In [ ]:
plot_data(input_data['AvgPrice'], 
          target_data['AvgPrice'], 
          "Input", 
          "Ideal Prediction",
          "Average Price unnormalized", 
          first_days=30)

We expect that both columns contain only numbers, thus we check if any `NaN`s or `None`s are present in the data.

In [ ]:
# Check if data is clean
# Check for NaN, None, and other non-numerical data
print("There are {} null values in input".format(input_data.isnull().values.sum()))
print("There are {} null values in target".format(target_data.isnull().values.sum()))

# In case any NaNs or Nones are found, we query for their indexes
columns = input_data.columns.values.tolist()
for column in columns:
    index = input_data[column].index[input_data[column].apply(np.isnan)]
    print('Index {} in column {} is NaN or Null'.format(index,column))

## Section 1.3: Divide Data into multiple sets

Now we divide the data into two sets: training and testing.

The training data set will be fed into network to train it.
After training is complete, the test data set will be used to evaluate how well the network has been trained.

### Datasets: Training, Validation, Test

**Note**: Usually the data is divided onto three datasets: training, validation and testing.

- `training` - used by the network in training,
- `validation` - used by the network for evaluation (but not for training),
- `test` - used by the user for evaluation

The process runs usually like this:

1. For each epoch do:
  1. For each batch in the training dataset do:
    1. Train the network using a batch
    2. Use validation data to evaluate the quality of prediction of the network
    3. Calculate loss (error) for training and validation
  2. Evaluate stop criterion: either STOP or start NEXT EPOCH
2. Finish training, store the model
3. Deploy the model and use the test data to evaluate it

Comparing the training loss against the validation loss allows to stop training at the right moment to prevent overfitting - i.e., learning by hard the right answers from the training data set.

Simple stopping criterion could look like follows: *continue training as long as both training and validation loss is decreasing. Stop training when training loss keeps decreasing while validation loss stops decreasing.*

### Our approach: Training + Test datasets

Due to a small data set, we do not use the `validation` data set. For this reason, we may not use the stopping criterion as depicted above. Instead, we will stop training after a fixed number of epochs (cycles). This makes our network prone to overfitting.


So lets, use first 95% of data as a training set and the last 5% of data as the testing set.

In [ ]:
def divide_stock_data(inputs, targets):
    """Prepare training and test sets from stock data."""
    
    data_len = len(inputs)
    # division point between train and test data
    ratio = 0.95
    division_index = int(data_len*ratio)
    
    inputs_train = inputs[:division_index]
    inputs_test = inputs[division_index:]
    
    targets_train = targets[:division_index]
    targets_test = targets[division_index:]
    
    return inputs_train, inputs_test, targets_train, targets_test

In [ ]:
train_X_orig, test_X_orig, train_y_orig, test_y_orig = divide_stock_data(input_data, target_data)
print("Stock days in data set: train = {}, test = {}".format(len(train_X_orig), len(test_X_orig)))

## Section 1.4: Scaling Data

Data should be normalized to simplify the learning process.
In this case, we use:
- `StandardScaler` to standardize features by removing the mean and scaling to unit variance
- `MinMaxScaler` to standardize features by placing all of them between 0 and 1


### Note!

We *fit* (configure) the scaler based on the training data, but we apply it to both: training and test.

We do this to realistically simulate that we have never seen the test data before - we assume that all that we have available at this stage is the training data.

In [ ]:
# Normalization of data

# .copy() prevents warnings when replacing single columns in DataFrames
# and allows to execute this cell multiple times without data overwritting
train_X = train_X_orig.copy()
train_y = train_y_orig.copy()
test_X  = test_X_orig.copy() 
test_y  = test_y_orig.copy() 

# We fit scalers using training data and then apply scaling to both training and test datasets
# We treat test data as we would never have seen it before

# scaler_price = StandardScaler()
scaler_price = MinMaxScaler()
scaler_price.fit(train_X[['AvgPrice']].values)

train_X[['AvgPrice']] = scaler_price.transform(train_X[['AvgPrice']])
train_y[['AvgPrice']] = scaler_price.transform(train_y[['AvgPrice']])

test_X[['AvgPrice']] = scaler_price.transform(test_X[['AvgPrice']])
test_y[['AvgPrice']] = scaler_price.transform(test_y[['AvgPrice']])

plot_data(train_X['AvgPrice'], train_y['AvgPrice'], "Input", "Target", "Average Price training data normalized. 60 days", 60)
plot_data(train_X['AvgPrice'], train_y['AvgPrice'], "Input", "Target", "Average Price training data normalized")

### Note! 
The plot of the test data below contains values higher than `1.0`.
This means that the test data set contains value that the network has never seen in the training.

In [ ]:
plot_data(test_X['AvgPrice'], test_y['AvgPrice'], "Input", "Target", "Average Price test data normalized")

# How to go back to the unscaled price/volume
denorm_X=scaler_price.inverse_transform(test_X[['AvgPrice']])
denorm_y=scaler_price.inverse_transform(test_y[['AvgPrice']])

plot_data(denorm_X, denorm_y, "Input", "Target", "Average Price test data denormalized (y-axis in €)")

Before we start with training, let's double-check if the input and target data are correctly reflected in arrays


In [ ]:
print("Ideal prediction done at day 0:")
print(train_y[0:1])
print("Should equal the average price from day 1:")
print(train_X[1:2])

print("\nThe same should hold for test data.")
print("Ideal prediction done at day 0:")
print(test_y[0:1])
print("Should equal the average price from day 1:")
print(test_X[1:2])

## Section 1.5: Save the data

Training and test data are packed into a csv files.

In [ ]:
data_dir = 'data'

pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join('data', 'train.csv'), header=False, index=False)


pd.concat([pd.DataFrame(test_y), pd.DataFrame(test_X)], axis=1) \
        .to_csv(os.path.join('data', 'test.csv'), header=False, index=False)

# Part 2: Local training and testing


## Section 2.1: Define a Neural Network Model

### Brief introduction

Neural network model can be seen as a Blackbox function that has inputs and outputs:

![Blackbox Neural Network](images/black-box1.png)

Each neural network has:
- a structure of its blackbox
- a set of parameters (weights) that configure the parts of its structure

Process of training consists of setting the weights in such a way, so that prediction error is minimized.

After the training is finished, the weights can be frozen and a trained model can be stored for use.

Using of the model consists of providing the input and reading the output.

### Model used in our case: LSTM

In our problem, we use a model consisting of LSTM (Long-short term memory) cells. **LSTM** is an example of recurrent neural network capable of learning long-term dependencies.

Recurrent neural networks allow to take parts of their outputs as inputs in order to learn from the history and maintain a hidden state. The hidden state is a parametrized memory where the parameters describing the short-term memory and long-term memory can be learnt in the learning process.


We define the inputs and outputs of the model as follows:

- **Input** is a vector of past average stock prices. The length of this vector in configurable and will be called `batch_size`. It contains data from day `0` to day `batch_size - 1`. Today is `data[batch_size - 1]`
- **Output** is also a vector of `batch_size` including average stock prices. It contains predictions for each elements of the input vector that are shifted by `forecast_days`, so that predictions for today are located in `predictions[batch_size - 2]` and for tomorrow in `predictions[batch_size - 1]` (under assumption that `forecast_days = 1`).

![Blackbox Neural Network with Inputs and Outputs](images/black-box2.png)

The Network looks conceptually as presented in Figure 3. Brief comments about the components:
- In this diagram, time flows from left to right,
- `A` block is a LSTM cell that takes input `x` and the state of the previous cell (right arrow between `A` blocks),
- `h` is a hidden state, which is also an output of the LSTM layer,
- LSTM layers can be stacked, so that output vector `h` can be the input to the next layer placed on top

![RNN concept](images/RNN-unrolled.png)

- *Image 3 source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

### Note!

We do not intend to go into details of LSTM. This is a relatively complex component.

We want to show the training and deployment in the AWS environment provided that a definition of the network is given e.g., taken from from a paper.

If you want to read more about LSTM, navigate to the following resources:
- Understanding LSTM Networks:   [https://colah.github.io/posts/2015-08-Understanding-LSTMs/](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) 
- LSTM processing animations:  [https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21) 

### Local training and testing

Before we use our model on AWS, we would like to try it locally to make sure that the implementation is bug-free and initial results are correct. For this, we will train and evaluate the model in this notebook.

First we will load the training data set to use. It may be time consuming to try and train the model completely in the notebook as we do not have access to a GPU and the compute instance that we are using is not particularly powerful. However, we can work on a portion the data to get a feel for how our training script is behaving.

(In this case, we will use entire data set, as the network that we use requires little computational power)

#### Model

The model is defined in `train/model.py` and the code looks as follows.

We se there the following elements:
- LSTM component, which can have multiple layers (parameter `self.num_layers`)
- Linear component that takes the output from LSTM (of size `self.hidden_dim`) and coverts it to the size `output_dim` which in our case is `1`
- function `init_hidden` that is responsible for initializing of the hidden layer in LSTM with zeros
- function `forward` that defines how to produce the `output` from the `input` to the network

In [ ]:
!pygmentize train/model.py

#### Read the training data and define size of the batch

In this block, we read the previously stored training data from a csv file.

We make sure that the number of samples from the training data set is a multiple of `batch_size`.

In [ ]:
# Read all data from file to calculate data length
dummy_read = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None)
num_rows_train = (len(dummy_read) // batch_size) * batch_size

# Read again so many rows, so that num_rows/batch_size is an integer
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=num_rows_train)

# Order of data in train.csv:
# Y-Average  |  X-Average  
# to_predict |  INPUT VALUE

# Turn the input in form of pandas.DataFrame into PyTorch Tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).float()

#print(train_sample)
print(train_sample_y.shape)
print(train_sample_X.shape)

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=batch_size)

### Writing the training method and testing it locally

Next we need to write the training code itself. We train the network in **epochs** and **batches**.

In a single *epoch*, the entire train dataset is fed to the network. We use multiple epochs for learning, so that the network can learn it better.

In a single *batch* a subset of the training data is presented to the network. 

Longer the batch means usually:
- better prediction results 
- less usability (more input is required to use the network)

The code below is commented to understand the process a bit better.

In [ ]:
# Import the model from the train/model.py file
from train.model import LSTM
   
def train(model, train_loader, epochs, optimizer, criterion, device):
    
    # For storing the prediction error over epochs
    loss_trace = []
    
    for epoch in range(1, epochs + 1):
        
        # Set model to train mode - allow gradient calculation and thus learning
        model.train()
        total_loss = 0
        num_batches = len(train_loader)
        
        # Iterate over batches
        for batch in train_loader:
            batch_X, batch_y = batch
            # Move data to GPU if available to speed-up calculations
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

            # Remove gradients from previous batches from the memory. Network has already learnt it
            model.zero_grad()
            
            # Obtain prediction for an input - run the `forward` function from the model definition
            output = model(batch_X)
           
            # Calculate prediction error a.k.a loss
            loss = criterion(output.squeeze(), batch_y)

            # Apply back-propagation step
            # Calculate corrections to the network, so that next time the error is (hopefully) smaller
            loss.backward()
            # Prevent `gradient explosion` phenomenon by clipping the gradients to a magic number
            nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_maximum)
            
            # Correct the weights of a network based on the corrections calculated previously
            # Here the learning takes place
            optimizer.step()
            
            # Collect prediction error for this batch
            total_loss += loss.data.item()
            
        avg_loss_per_batch = total_loss/num_batches
        print("Epoch: {}, MSELoss per batch: {}".format(epoch, avg_loss_per_batch))
        loss_trace.append(avg_loss_per_batch)
    return model, loss_trace

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: {}".format(device))
model = LSTM(input_dim=1, hidden_dim=24, batch_size=batch_size, output_dim=1, num_layers=2).to(device)

print(model)


optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()

model, loss_trace = train(model, train_sample_dl, 15, optimizer, loss_fn, device)

In [ ]:
np_loss_trace = np.array(loss_trace).reshape(-1, 1)

plot_data(np_loss_trace, None, "Loss", "",
          "Avergage loss per batch for training epochs", len(loss_trace))

Supposing we have the training method above, we will test that it is working by writing a bit of code in the notebook that executes our training method on the small sample training set that we loaded earlier. The reason for doing this in the notebook is so that we have an opportunity to fix any errors that arise early when they are easier to diagnose.

In [ ]:
test_dummy = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None, names=None)

num_rows_test = (len(test_dummy) // batch_size) * batch_size

test_sample = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None, names=None, nrows=num_rows_test)
test_y = torch.from_numpy(test_sample[[0]].values).float().squeeze()

test_X = torch.from_numpy(test_sample.drop([0], axis=1).values).float()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size)




def test(model, test_loader, criterion, device):
    
    model.eval()
    
    data = dict()
    data['prediction'] = []
    data['expected'] = []

    total_loss = 0
    total_rmse = 0
    num_batches = len(test_loader)

    for batch in test_loader:
        batch_X, batch_y = batch
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
    
        output = model(batch_X)
        loss = criterion(output.squeeze(), batch_y)
        
        # Calculate errors - note copying data from GPU to CPU!
        flat_output = output.cpu().detach().numpy().reshape(-1,1)              
        flat_y = batch_y.cpu().detach().numpy()
        data['prediction'].append(flat_output)
        data['expected'].append(flat_y)
        total_rmse += mean_squared_error(flat_y, flat_output)
        total_loss += loss.data.item()

        
    print("test:mseloss: {}".format(total_loss/num_batches))
    print("test:rmse: {}".format(total_rmse/num_batches))
    return data

loss_fn = torch.nn.MSELoss()
result = test(model, test_dl, loss_fn, device)

### Displaying prediction results

Note that Y-axis is still normalized and some values are $> 1$.
This means that they are `out of scale`, and the predictor will have problems with precise prediction in such cases, because it have never seen such training data before.

In [ ]:
# Remove batching from result
predictions = []
for batch_arr in result['prediction']:
    for value in batch_arr:
        predictions.append(value)

expectations = []
for batch_arr in result['expected']:
    for value in batch_arr:
        expectations.append(value)

plot_data(np.array(expectations).reshape(-1, 1), 
          np.array(predictions).reshape(-1, 1), 
          "Expected prediction result",
          "Obtained prediction result",
          "Test data: prediction vs expectation - normalized", len(expectations))

### Invert scaling transformation

Let's look at the chart with Y-axis expressed in euro

In [ ]:
exp = scaler_price.inverse_transform(np.array(expectations).reshape(-1, 1))
pred = scaler_price.inverse_transform(np.array(predictions).reshape(-1, 1))

plot_data(exp, pred, 
          "Expected prediction result",
          "Obtained prediction result",
          "Test data: prediction (red) expectation (blue) - denormalized in €", len(expectations))

# Part 3: AWS

## Section 3.1: Prepare Custom Code for Training and Prediction

### Model as defined by Amazon SageMaker

In particular, a model comprises three objects

 - Model Artifacts (provided in `train/model.py`),
 - Training Code (provided in `train/train.py`),
 - Inference Code (also provided in `train/train.py`).
 
each of which interact with one another. Here we will still be using containers provided by Amazon with the added benefit of being able to include our own custom code.

You already have seen the model in `train/model.py`. Parts of the training code were also used for local training. 
Let's peek into the training and inference code defined in `train/train.py`.

In [ ]:
!pygmentize train/train.py

## Section 3.2: Uploading the training data to S3


Next, we need to upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

In [ ]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/stock-predictor-dojo1'

role = sagemaker.get_execution_role()

We upload entire data directory containing `train.csv` and `test.csv` files.

In [ ]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

## Section 3.3: Train the model on a separate GPU-enabled EC2 instance

In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

We may also configure the type of EC2 instance that will be used for training. Here, we use one of the `p2.*` instances, which are GPU-enabled. We want to minimize the time we use the `p2` instance, because they are relatively expensive - cheapest one costs about $1 per hour.

from sagemaker.pytorch import PyTorch
Once a model is trained, we usually need much less computing power to use the model. Thus, we will later deploy the trained model on a weaker, non-GPU instance `m4.xlarge`.

In [ ]:
instance_type_training = 'ml.p2.xlarge'
instance_type_deployment = 'ml.m4.xlarge'
# Must be a 'ml.*'instance, for example:
# - ml.m4.xlarge - new EC2 instance with CPU
# - ml.p2.xlarge - new EC2 instance with GPU

### Custom Metrics

We may define custom metrics that AWS will parse and present in CloudWatch.
The output from training based on a regular expression and present the data as a chart in CloudWatch.

In [ ]:
# Custom metrics
metric_definitions = [{'Name': 'Training-Loss',
                       'Regex': 'train:mseloss: ([0-9\\.]+)'}]

### Defining an Estimator

Next, we define an `estimator` object with custom entry point (reference to our custom code) and optionally custom metrics definitions.

In [ ]:
estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type=instance_type_training,
                    metric_definitions = metric_definitions,
                    hyperparameters={
                        'epochs': 50,
                        'hidden-dim': 100,
                        'batch-size': batch_size,
                        'num-layers': 4,
                        'learning-rate': 0.0001 # Keep not higher than 0.0002
                    })

### Training

Next, we instruct to fit the estimator. Fitting is a better term to describe model learning :)

In [ ]:
# Training with given hyperparameters
estimator.fit({'training': input_data})

Note the `Billable seconds: XX` output -  this is the total cost of running the training instance. 
This allows to run powerful multi-GPU instances at the same time minimizing the AWS costs.

By navigating to training jobs (https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs) one can see various details like: monitoring of resources usage, potential errors, etc.

The trained model is stored in S3, so that it can be deployed and used in a predictor later.

## Section 3.4: Deploy the Trained Model on a Separate Instance

In order to use the model over a REST endpoint, we need to deploy it first.

Deployed predictor is an EC2 instance holding the trained model. 
It can be accessed via `endpoint`. All endpoints are listed under: https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/endpoints

In [ ]:
# Deploy the trained model
predictor = estimator.deploy(initial_instance_count=1, 
                         instance_type=instance_type_deployment)

## Section 3.5: Use the Deployed Model for Predictions

Now the model is ready for use! Let's predict the price for tomorrow then!

In [ ]:
# Let's get prices of SAP stocks from the last batch_size days
test_data = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None, names=None)
test_input = np.array(test_data[1])
test_expected_result = np.array(test_data[0])

# Slice the last batch_size days out of the test data
input_1 = np.array(test_input[-1*batch_size:])
expected_result_1 = np.array(test_expected_result[-1*batch_size:])

print("Input to the predictor - scaled average prices from the last {} days:\n{}".format(batch_size,input_1))

# Predict the prices
result = predictor.predict(input_1)

# Change the shape of the result from (1,X,1) to (X,1)
result = np.array(result).reshape(-1, 1)

# Denormalize the data, i.e., convert back to euro:
scaled_result = scaler_price.inverse_transform(result[-1:]).item()
print("Based on the trend from the last {} days, the price tomorrow will be: {}".format(batch_size, scaled_result))

input_scaled = scaler_price.inverse_transform(input_1.reshape(-1, 1))
result_scaled = scaler_price.inverse_transform(result)
expected_result_scaled = scaler_price.inverse_transform(expected_result_1.reshape(-1, 1))

plot_data(input_scaled, 
          result_scaled, 
          "Input - real price",
          "Prediction - price next day",
          "Input and output of the network - in €", 
          len(input_scaled))

plot_data(expected_result_scaled, 
          result_scaled, 
          "Ideal prediction - price next day",
          "Obtained prediction - price next day",
          "Prediction quality - in €", 
          len(result_scaled))


### Testing the deployed model on the test data set

Now, let's provide all batches from the test data set as input and analyze the prediction quality.

Note, that we provide the network with the data that has not been used in training.

In [ ]:
def predict(data, rows=batch_size):
    predictions = np.array([])
    
    batched = data.view(-1, batch_size)
    for batch in batched:
        result = predictor.predict(batch.numpy())
        predictions = np.append(predictions, np.array(result))
    
    return predictions

In [ ]:
# Test using all batches from the testing data set
predictions = predict(test_X, batch_size)
predictions = [num for num in np.array(predictions)]

In [ ]:
print("mean_squared_error when all predcitions = 1.0: ", mean_squared_error(test_y, np.ones(300)))
print("mean_squared_error when all predcitions = 0.0: ", mean_squared_error(test_y, np.zeros(300)))
print("mean_squared_error obtained result: ", mean_squared_error(test_y, np.array(predictions)))

In [ ]:
target_scaled = scaler_price.inverse_transform(test_y.numpy().reshape(-1, 1))
result_scaled = scaler_price.inverse_transform(np.array(predictions).reshape(-1, 1))

plot_data(target_scaled, 
          result_scaled, 
          "Ideal price prediction",
          "Obtained price prediction",
          "Test data: grasp on prediction quality - in €)", 
          len(target_scaled))


### Testing the deployed model on the **training** data set

We may check if the model really well learned the data that was used for training.
We expect here a bit of overfitting (learning the training data), so the prediction should be much better here.

In [ ]:
# Check how the predictor performs on the original training set

train_data = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None)
train_input = np.array(train_data[1])
train_expectation = np.array(train_data[0])

input_2 = np.array(train_input[-1*batch_size:])
train_expectation = np.array(train_expectation[-1*batch_size:].reshape(-1,1))

predictions_train = predictor.predict(input_2)
predictions_train = [num for num in np.array(predictions_train)]
predictions_train = np.array(predictions_train).reshape(-1,1)

print("mean_squared_error: ", mean_squared_error(train_expectation, predictions_train))

target_scaled_train = scaler_price.inverse_transform(train_expectation)
result_scaled_train = scaler_price.inverse_transform(predictions_train)

plot_data(target_scaled_train, 
          result_scaled_train, 
          "Ideal price prediction",
          "Obtained price prediction",
          "Training data: grasp on prediction quality - in €)", 
          len(target_scaled))

In [ ]:
# If you forget to delete the endpoint, the EC2 instance will be running and generating costs
predictor.delete_endpoint()